In [ ]:
## inputs are the initial condition and the edge matrix
def find_tranformation_matrix_naive(b, E):

    # ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    ## number of non-zero rows
    z = np.ones(N, dtype=bool)

    # run simulation
    for t in range(n):
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False 
        ## check if it is an all zero row by checking correpsonding entry in Z, if yes, nothing to be done; else, zero row and set corresponding entry in Z to False
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")

            if z[i] == True:

                X[i, :] = False

                z[i] == False

        else:
        ## spreading event (turns symmetric entries to 1 and premultiplies A) == row i and j of A is set to the OR of row i and j
        ## check if the two rows being ORed are all zero rows, if yes, nothing to be done; else, OR the rows and set corresponding entries in Z to True

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if z[i] == z[j] == False:

                pass

            else:

                X[i] = X[j] = np.logical_or(X[i], X[j])

                z[i] = z[j] = True
        
        ## check for each iteration of the alogrithm if the number of distinct columns is 2
        distinct_rows = np.unique(X, axis=0).shape[0]
        distinct_cols = np.unique(X, axis=1).shape[1]
        
        ## check to ensure that disctinct columsn == distinct rows (this is true)
        # print(distinct_rows)
        # print(distinct_cols)

        if distinct_cols == 2:
            break

    return X



In [157]:
import numpy as np
# network size N
N = 5

# number of edges
M = N*3

## initialize boolean identity Matrix A of dimensions NxN
X = np.identity(N, dtype=bool)

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

## number of non-zero rows
z = np.ones(N, dtype=bool)

distinct_rows = N


There are two ways to check for distinct rows, I'm unsure which is faster https://stackoverflow.com/questions/14766194/testing-whether-a-numpy-array-contains-a-given-row says that it is variable

In [169]:
i,j = E[np.random.randint(M)]

if z[i] or z[j] == True:

## nothing happens if two rows being ORed are two zero rows

    if np.array_equal(X[i], X[j]) == False:

        print(f"spreading event at iteration {1} of nodes {i} and {j}")
                    
        ## no change in distinct rows if the two rows being ORed are the same
        # have confirmed that row A and row B are not the same and at least one of them is nonzero
        X_test = np.delete(X, [i, j], 0)
        new_not_distinct = any(np.equal(X_test,np.logical_or(X[i], X[j])).all(1))
        print(new_not_distinct)
        row_i_not_distinct = any(np.equal(X_test,X[i]).all(1))
        print(row_i_not_distinct)
        row_j_not_distinct = any(np.equal(X_test,X[j]).all(1))
        print(row_j_not_distinct)

        ## if the row being created is distinct, but the rows being collapsed were also distinct, then the number of distinct rows is one less
        # if the row being created is distinct, but ONE of the rows being collapsed was distinct, then the number of distinct rows is the same
        ## if the row being created is distinct, but NEITHER of the rows collapsed were distinct, then the number of distinct rows is increasing by 1
        if new_not_distinct == False:
            if row_i_not_distinct == False and row_j_not_distinct == False:
                print("we're subtracting 1")
                distinct_rows -= 1
            if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                    print("we're keeping distinct rows the same")
                    distinct_rows = distinct_rows
            if row_i_not_distinct == True and row_j_not_distinct == True:
                    print("we're adding 1")
                    distinct_rows = min(distinct_rows + 1, N)
                    ## if the row being created is not distinct, but BOTH the rows being collapsed were, then the number of distinct rows decreases by 2
                    ## if the row being created is not distinct, but ONE of the rows being collapsed was, then the number of distinct rows decreases by 1
                    ## if the row being created is not distinct, but NEITHER of the rows being collapsed were, then the number of distinct rows remains the same
        else:
            if row_i_not_distinct == False and row_j_not_distinct == False:
                print("we're subtracting 2")
                distinct_rows -= 2
            if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                print("we're subtracting 1")
                distinct_rows -= 1
                    
        ## progres the simulation normally
        X[i] = X[j] = np.logical_or(X[i], X[j])

        z[i] = z[j] = True
    
    else:
        print("we have not done anything this iteration")
                
    print(distinct_rows)    


we have not done anything this iteration
2


In [64]:
distinct_rows = 4
new_not_distinct = True
row_i_not_distinct = True 
row_j_not_distinct = True 

if new_not_distinct == False:
    if row_i_not_distinct == False and row_j_not_distinct == False:
        print("we're subtracting 1")
        distinct_rows -= 1
    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
            print("we're keeping distinct rows the same")
            distinct_rows = distinct_rows
    if row_i_not_distinct == True and row_j_not_distinct == True:
        print("we're adding 1")
        distinct_rows = min(distinct_rows + 1, N)
else:
    if row_i_not_distinct == False and row_j_not_distinct == False:
        print("we're subtracting 2")
        distinct_rows -= 2
    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
        print("we're subtracting 1")
        distinct_rows -= 1
print(distinct_rows)


4


One way to check if the rows are distinct.

In [151]:
i = 2
j = 3
X = np.array([[False, False, False, False, False], [False, True, False, False, False], [False, False, True, False, False], [False, False, False, True, False], [False, False, False, False, True]])
X_test = np.delete(X, [i, j], 0)
print(X_test)
new_not_distinct = np.logical_or(X[i], X[j]).tolist in X_test.tolist()
print(new_not_distinct)
row_i_not_distinct = X[i].tolist() in X_test.tolist()
print(row_i_not_distinct)
row_j_not_distinct = X[j].tolist() in X_test.tolist()
print(row_j_not_distinct)


[[False False False False False]
 [False  True False False False]
 [False False False False  True]]
False
False
False


Another way to check if the row is distinct!

In [156]:
i = 2
j = 3
X = np.array([[False, False, False, False, False], [False, True, False, False, False], [False, False, True, False, False], [False, False, False, True, False], [False, False, False, False, True]])
X_test = np.delete(X, [i, j], 0)
print(X_test)
# new_not_distinct = np.logical_or(X[i], X[j]).tolist in X_test.tolist()
# print(new_not_distinct)
row_i_not_distinct = any(np.equal(X_test,X[i]).all(1))
print(row_i_not_distinct)
row_j_not_distinct = any(np.equal(X_test,X[j]).all(1))
print(row_j_not_distinct)
new_not_distinct = any(np.equal(X_test,np.logical_or(X[i], X[j])).all(1))
print(new_not_distinct)



# row_j_not_distinct = X[j].tolist() in X_test.tolist()
# print(row_j_not_distinct)


[[False False False False False]
 [False  True False False False]
 [False False False False  True]]
False
False
False


The number of distinct columns and rows are not always equal. But in our example since only the changes that take place are constrained; they will be. 

In [129]:
X = np.array([[False, False, True, False, False], [False, True, False, False, False], [False, False, True, False, False], [False, False, False, True, False], [False, False, False, False, True]])
print(X)
distinct_rows = np.unique(X, axis=0).shape[0]
distinct_cols = np.unique(X, axis=1).shape[1]
print(distinct_rows)
print(distinct_cols)


[[False False  True False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]]
4
5


In [179]:
# network size N
N = 5

# number of edges
M = N*3

Tmax = 5

## inputs are the initial condition and the edge matrix
def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))
    print(n)
    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)

    ## initialize the number of distinct rows to N (start with Identity matrix)
    distinct_rows = N

    # run simulation
    for t in range(n):
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False 
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")

            # if z[i] == True:
                
            X_test = np.delete(X, i, 0)

            row_i_not_distinct = any(np.equal(X_test,X[i]).all(1))

            zero_row = any(np.equal(np.zeros(N), X_test).all(1))

            X[i, :] = False

            if zero_row == True:
                    ## if there is at least one non zero row, and the row being zeroed was distinct, then the number of distinct rows decreases by 1
                if row_i_not_distinct == False:

                    distinct_rows -= 1
                    ## if there is at least one non zero row and the row being zeroed was not distinct, then the number of distinct rows remains the same
                     
            else:
                    ## if there are no nonzero rows, and the row being zeroed was not distinct, then the number of distinct rows increases by 1
                if row_i_not_distinct == True:
                            
                    distinct_rows = min(distinct_rows + 1, N)
                    ## if there are no nonzero rows, and the row being zeroed was distinct, then the number of distinct rows remains the same

        else:
        ## spreading event (turns symmetric entries to 1 and premultiplies A) == row i and j of A is set to the OR of row i and j
        ## check if the two rows being ORed are all zero rows, if yes, nothing to be done; else, OR the rows and set corresponding entries in Z to True

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if z[i] or z[j] == True:

                ## nothing happens if two rows being ORed are two zero rows

                if np.array_equal(X[i], X[j]) == False:
                    
                    ## no change in distinct rows if the two rows being ORed are the same
                    # have confirmed that row A and row B are not the same and at least one of them is nonzero
                    X_test = np.delete(X, [i, j], 0)
                    new_not_distinct = any(np.equal(X_test,np.logical_or(X[i], X[j])).all(1))
                    print(new_not_distinct)
                    row_i_not_distinct = any(np.equal(X_test,X[i]).all(1))
                    print(row_i_not_distinct)
                    row_j_not_distinct = any(np.equal(X_test,X[j]).all(1))
                    print(row_j_not_distinct)

                    ## if the row being created is distinct, but the rows being collapsed were also distinct, then the number of distinct rows is one less
                    # if the row being created is distinct, but ONE of the rows being collapsed was distinct, then the number of distinct rows is the same
                    ## if the row being created is distinct, but NEITHER of the rows collapsed were distinct, then the number of distinct rows is increasing by 1
                    if new_not_distinct == False:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 1
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows = distinct_rows
                        if row_i_not_distinct == True and row_j_not_distinct == True:
                            distinct_rows = min(distinct_rows + 1, N)
                    ## if the row being created is not distinct, but BOTH the rows being collapsed were, then the number of distinct rows decreases by 2
                    ## if the row being created is not distinct, but ONE of the rows being collapsed was, then the number of distinct rows decreases by 1
                    ## if the row being created is not distinct, but NEITHER of the rows being collapsed were, then the number of distinct rows remains the same
                    else:
                        if row_i_not_distinct == False and row_j_not_distinct == False:
                            distinct_rows -= 2
                        if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                            distinct_rows -= 1
                    
                ## progres the simulation normally
                X[i] = X[j] = np.logical_or(X[i], X[j])

                z[i] = z[j] = True
                      
        print(distinct_rows)    
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)
        if distinct_rows == 2:
            
            break

    return X



In [181]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)

matrix_A = find_tranformation_matrix(b, E)

print(matrix_A)


61
spreading event at iteration 0 of nodes 3 and 1
False
False
False
4
recovery event at iteration 1 of node 4
4
spreading event at iteration 2 of nodes 4 and 1
True
False
True
3
spreading event at iteration 3 of nodes 2 and 3
False
False
True
3
recovery event at iteration 4 of node 4
4
recovery event at iteration 5 of node 2
4
recovery event at iteration 6 of node 3
3
spreading event at iteration 7 of nodes 1 and 2
False
False
True
3
recovery event at iteration 8 of node 1
3
spreading event at iteration 9 of nodes 2 and 3
False
False
True
3
recovery event at iteration 10 of node 1
3
spreading event at iteration 11 of nodes 0 and 1
False
False
True
3
spreading event at iteration 12 of nodes 1 and 2
False
True
True
4
spreading event at iteration 13 of nodes 3 and 2
True
False
True
3
spreading event at iteration 14 of nodes 2 and 4
True
True
False
2
[[ True False False False False]
 [ True  True False  True False]
 [ True  True False  True False]
 [ True  True False  True False]
 [ True 